In [19]:
import os

from dotenv import load_dotenv
from langchain_community.tools import WikipediaQueryRun
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_community.utilities import WikipediaAPIWrapper
from smolagents import CodeAgent, OpenAIServerModel, Tool, ToolCallingAgent

load_dotenv()

True

In [20]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
wikipedia.run("HUNTER X HUNTER")

'Page: Hunter × Hunter\nSummary: Hunter × Hunter (pronounced "hunter hunter") is a Japanese manga series written and illustrated by Yoshihiro Togashi. It has been serialized in Shueisha\'s shōnen manga magazine Weekly Shōnen Jump since March 1998, although the manga has frequently gone on extended hiatuses since 2006. Its chapters have been collected in 38 tankōbon volumes as of September 2024. The story focuses on a young boy named Gon Freecss who discovers that his father, who left him at a young age, is actually a world-renowned Hunter, a licensed professional who specializes in fantastical pursuits such as locating rare or unidentified animal species, treasure hunting, surveying unexplored enclaves, or hunting down lawless individuals. Gon departs on a journey to become a Hunter and eventually find his father. Along the way, Gon meets various other Hunters and encounters the paranormal.\nHunter × Hunter was adapted into a 62-episode anime television series by Nippon Animation and d

In [21]:
wikipedia_tool = Tool.from_langchain(wikipedia)

In [22]:
yahoo_finance = YahooFinanceNewsTool()
yahoo_finance.run("Meta")

'No news found for company that searched with Meta ticker.'

In [23]:
yahou_finance_tool =  Tool.from_langchain(yahoo_finance)

In [24]:
web_search_agent = ToolCallingAgent(
    tools=[wikipedia_tool, yahou_finance_tool],
    model=OpenAIServerModel(
        model_id="gpt-4o",
        api_key=os.getenv("OPENAI_API_KEY"),
    ),
    max_steps=10,
    name="web_search_agent",
    description="This agent can search wikipedia and Yahoo Finance to get financial information about a company.",
)

In [25]:
web_search_agent.run(task="meta last results")

╭────────────────────────────────────────── New run - web_search_agent ───────────────────────────────────────────╮
│                                                                                                                 │
│ meta last results                                                                                               │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'yahoo_finance_news' with arguments: {'query': 'META'}                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: No news found for company that searched with META ticker.

[Step 1: Duration 15.24 seconds| Input tokens: 1,073 | Output tokens: 16]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "There are currently no financial news articles         │
│ available for the company with the ticker 'META'."}                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: There are currently no financial news articles available for the company with the ticker 'META'.

[Step 2: Duration 2.20 seconds| Input tokens: 2,242 | Output tokens: 47]

"There are currently no financial news articles available for the company with the ticker 'META'."

In [26]:
planner_agent = CodeAgent(
    model=OpenAIServerModel(model_id="gpt-4o"),
    tools=[],
    managed_agents=[web_search_agent],
    additional_authorized_imports=["time", "numpy", "pandas"],
    description="This agent can search wikipedia and Yahoo Finance, and compute rentability KPIs for stocks using Python scripts.",
)

In [27]:
planner_agent.run("Tell me the rentability of Meta stocks in the last 3 months, and give me a Python script to compute it.")


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tell me the rentability of Meta stocks in the last 3 months, and give me a Python script to compute it.         │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\s*\n(.*?)\n``` was not found in it.
            Here is your code snippet:
            To determine the rentability of Meta stocks over the last 3 months, we need to follow these steps:
1. Obtain the stock price of Meta from 3 months ago.
2. Obtain the current stock price of Meta.
3. Calculate the rentability using the formula: 

   \[
   \text{Rentability} = \left( \frac{\text{Current Price} - \text{Price 3 Months Ago}}{\text{Price 3 Months Ago}} 
\right) \times 100
   \\]

I will use the `web_search_agent` to find both current and historical stock prices for Meta. Then, I'll provide the
Python script for the rentability calculation.

Let's begin by searching for the current stock price and the stock price from 3 months ago.
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            Code:
            ```py
            # Your python code here
            ```<end_code>
Make sure to provide correct code blobs.

[Step 1: Duration 4.36 seconds| Input tokens: 2,091 | Output tokens: 161]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

Execution logs:
Price 3 months ago: 300.0
Current price: 350.0
Rentability over the last 3 months: 16.666666666666664 %

Out: None

[Step 20: Duration 16.40 seconds| Input tokens: 126,394 | Output tokens: 3,483]

Reached max steps.

[Step 21: Duration 17.38 seconds| Input tokens: 134,734 | Output tokens: 3,772]

'To determine the rentability of Meta stocks over the past 3 months, you can use the following Python script. This script calculates rentability based on hypothetical stock prices, which you\'ll need to replace with actual data to perform a real analysis. Here is the script:\n\n```python\n# Python script to compute the rentability of Meta stocks.\n# Update these variables with real stock prices to find the actual rentability.\n\nprice_3_months_ago = 300.00  # Replace with actual stock price 3 months ago\ncurrent_price = 350.00       # Replace with today\'s stock price\n\n# Calculate rentability\nrentability = ((current_price - price_3_months_ago) / price_3_months_ago) * 100\n\n# Output the calculated results\nprint("Price 3 months ago:", price_3_months_ago)\nprint("Current price:", current_price)\nprint("Rentability over the last 3 months:", rentability, "%")\n```\n\nUsing the hypothetical data (e.g., a stock price of 300.00 three months ago and 350.00 currently), the rentability is ap